In [ ]:
from cedne import utils
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import matplotlib
import matplotlib.cm as cm

In [ ]:
jsons = {}
for js in os.listdir('/Users/sahilmoza/Documents/Postdoc/Yun Zhang/data/SteveFlavell-NeuroPAL-Cell/Control/'):
    with open ("/Users/sahilmoza/Documents/Postdoc/Yun Zhang/data/SteveFlavell-NeuroPAL-Cell/Control/{}".format(js), 'r') as f:
        jsons['Atanas et al (2023) ' +  js] = json.load(f)

In [ ]:
w = utils.makeWorm('atanas')
nn = w.networks['Neutral']

In [ ]:
measuredNeurons = {}
for js, p in jsons.items():
    sortedKeys = sorted ([int(x) for x in (p['labeled'].keys())])
    labelledNeurons = {p['labeled'][str(x)]['label']:x for x in sortedKeys if not '?' in p['labeled'][str(x)]['label']} # Removing unsure hits
    measuredNeurons[js] = {m:i for i,m in enumerate(set(labelledNeurons))}

In [ ]:
jsons.keys()
database = 'Atanas et al (2023) 2022-06-14-01.json'

In [ ]:
timepoint = 100
for neuron in nn.neurons:
    if neuron in measuredNeurons[database]: 
        nn.neurons[neuron].set_property('amplitude', jsons[database]['trace_array'][measuredNeurons[database][neuron]][timepoint])

In [ ]:
utils.loadSynapticWeights(nn)

In [ ]:
nodeColors = {}
nodelist = []
for neuron in nn.neurons:
    if neuron in measuredNeurons[database]:
        nodeColors[neuron] = nn.neurons[neuron].amplitude
        nn.neurons[neuron].set_property('color', nn.neurons[neuron].amplitude)
        nodelist.append(neuron)

In [ ]:
nn_2 = nn.subnetwork(nodelist, as_view=False)

In [ ]:
## Giving the sex specific neurons an interneuron type for positioning on graph.
sex_neurons = ['CANL', 'CANR']
for n in nn_2.neurons:
    if n in sex_neurons:
        nn_2.neurons[n].type = 'interneuron'

In [ ]:
edgeColors = []
for e in nn_2.connections:
    edgeColors.append(nn_2.connections[e].weight)
cmap = plt.get_cmap('PuOr')
max_color = max(np.abs(edgeColors))
norm = matplotlib.colors.Normalize(vmin=-max_color,vmax=max_color)
m = cm.ScalarMappable(norm=norm, cmap=cmap)
edge_color_dict = {e: m.to_rgba(nn_2.connections[e].weight) for e in nn_2.connections}


max_color = max(np.abs(list(nodeColors.values())))
cmap2 = plt.get_cmap('RdYlGn')
norm2 = matplotlib.colors.Normalize(vmin=-max_color,vmax=max_color)
o = cm.ScalarMappable(norm=norm2, cmap=cmap2)

node_color_dict = {nn_2.neurons[n]: o.to_rgba(nn_2.neurons[n].color) for n in nn_2.neurons}

In [ ]:
center = [n for n in nn_2.neurons if nn_2.neurons[n].type == 'sensory']
utils.plot_shell(nn_2, center=center, figsize=(10,10), edge_color_dict=edge_color_dict, node_color_dict=node_color_dict, save=False)#"weight-activity.pdf")

In [ ]:
conns = [(e[0].name, e[1].name) for e in nn_2.connections]
edgeColors = [nn_2.connections[e].weight for e in nn_2.connections]

utils.plot_layered(conns, nn_2, nodeColors=nodeColors, edgeColors=edgeColors)

In [ ]:
conns = [(e[0].name, e[1].name) for e in nn_2.connections]

In [ ]:
nn_2.edges

In [ ]:
for n in nn_2.neurons:
    print(n, nn_2.neurons[n].type, nn_2.neurons[n].category)

In [ ]:
import pandas as pd


TOPDIR = '../../' ## Change this to cedne and write a function to download data from an online server for heavy data.
DATADIR = TOPDIR + 'data_sources/'
DOWNLOAD_DIR = TOPDIR + 'data_sources/downloads/'

prefix_NT = 'Wang_2019/'
prefix_CENGEN = 'CENGEN/'
prefix_NP = 'Ripoll-Sanchez_2023/'
prefix_synaptic_weights = 'Randi_2023/'
weightMatrix = DOWNLOAD_DIR + prefix_synaptic_weights + "41586_2023_6683_MOESM13_ESM.xls"
tMat = pd.read_excel(weightMatrix, index_col=0).T

In [ ]:
for e in nn.neurons['AWCL'].out_connections:
    if(nn.neurons['AWCR'] in e):
        print(nn.connections[e])

In [ ]:
for n in nn_2.neurons: 
    print(nn_2.neurons[n].type)
    print(nn_2.neurons[n].category)

In [ ]:

deColors